In [111]:
import pandas as pd

In [112]:
df = pd.read_csv('Streamlit/films_data.csv')
df.head(3)

,page_url,image_url,movie_title,year,description,director,actors,genre,rating
0,https://www.film.ru/movies/malholland-drayv,https://www.film.ru/sites/default/files/styles...,Малхолланд драйв,2001,"Чувственная красавица-брюнетка, чудом уцелевша...",Дэвид Линч,"Наоми Уоттс, Жанна Бэйтс, Дэн Бирнбаум, Лора Х...","детектив, драма, триллер",7.9
1,https://www.film.ru/movies/kradushchiysya-tigr...,https://www.film.ru/sites/default/files/styles...,"Крадущийся тигр, затаившийся дракон",2000,Великий воин Ли Му Бай вдруг оказался поколебл...,Энг Ли,"Чоу Юньфат, Мишель Йео, Чжан Цзыи, Чан Чень","боевик, драма, приключения",7.9
2,https://www.film.ru/movies/proshlye-zhizni,https://www.film.ru/sites/default/files/styles...,Прошлые жизни,2023,Действие картины разворачивается в трех разных...,Селин Сон,"Грета Ли, Тео Ю, Джон Магаро, Moon Seung-ah","драма, мелодрама",7.8


In [113]:
df = df.dropna()
df.isna().sum()

page_url       0
image_url      0
movie_title    0
year           0
description    0
director       0
actors         0
genre          0
rating         0
dtype: int64

In [114]:
df['rating'] = pd.to_numeric(df['rating'], errors='coerce').fillna(0.0)

In [115]:
df.dtypes

page_url        object
image_url       object
movie_title     object
year             int64
description     object
director        object
actors          object
genre           object
rating         float64
dtype: object

In [116]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [117]:
client = QdrantClient(
    path='db/qdrant_db'
)

In [118]:
client.create_collection(
    collection_name='demo_collection',
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

True

In [119]:
from langchain_huggingface import HuggingFaceEmbeddings

In [120]:
model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
model_kwargs = {'device': 'mps'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [121]:
from uuid import uuid4
from langchain_core.documents import Document

In [122]:
documents = []
id_mapping = {}


for _, row in df.iterrows():

    content = row['description']

    genre_raw = row.get('genre', '')
    genre_list = [g.strip().lower() for g in str(genre_raw).split(',') if g.strip()]

    director_raw = row.get('director', '')
    director_list = [d.strip() for d in str(director_raw).split(',') if d.strip()]

    actors_raw = row.get('actors', '')
    actors_list = [a.strip() for a in str(actors_raw).split(',') if a.strip()]

    metadata = {
        'movie_title': row.get('movie_title', ''),
        'year': int(row.get('year', 0)),
        'director': director_list,
        'actors': actors_list,
        'genre': genre_list,
        'rating': row.get('rating', 0),
        'page_url': row.get('page_url', ''),
        'image_url': row.get('image_url', '')
    }

    documents.append(Document(page_content=content, metadata=metadata))

uuids = [str(uuid4()) for _ in range(len(documents))]

print(f'✅ Создано {len(documents)} документов с UUID')

✅ Создано 5257 документов с UUID


In [123]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name='demo_collection',
    embedding=hf
)

In [124]:
from tqdm import tqdm

In [125]:
with tqdm(total=len(documents), desc='Добавление документов в Qdrant') as pbar:
    for doc, doc_id in zip(documents, uuids):
        vector_store.add_documents(documents=[doc], ids=[doc_id])
        pbar.update(1)

print(f"✅ {len(documents)} документов добавлено в Qdrant!")

Добавление документов в Qdrant: 100%|██████████| 5257/5257 [02:28<00:00, 35.48it/s]

✅ 5257 документов добавлено в Qdrant!


In [126]:
points, _ = client.scroll(
    collection_name='demo_collection',
    limit=2,
    with_payload=True,
    with_vectors=True
)
points

[Record(id='0019581a-ef1e-4e9a-a06c-fc246b72fa8c', payload={'page_content': 'Репортер Саймон Хант и оператор Дак вместе побывали в самых горячих точках планеты: от Боснии до Ирака, от Сомали до Эль Сальвадора. Они вместе уклонялись от пуль, записывали впечатляющие сюжеты и получали награды Эмми. Но однажды все изменилось. Во время прямого эфира из одной Боснийской деревушки с Саймоном случается нервный приступ, после чего Дака повышают по службе, а Саймон куда-то исчезает. Спустя пять лет Дак возвращается в Сараево с начинающим репортером Бенджамином, чтобы сделать репортаж, посвященный пятилетию окончания войны. Внезапно появляется Саймон и обещает им эксклюзивный сенсационный репортаж. Он убеждает Дака, что знает, где найти "Лиса" – Боснийского преступника номер один, которого разыскивают все спецслужбы мира. С этой опасной миссией трое журналистов, вооруженные лишь невнятной информацией, отправляются в самую глубь враждебной территории.', 'metadata': {'movie_title': 'Охота Ханта', '

In [127]:
points[0].payload['metadata']

{'movie_title': 'Охота Ханта',
 'year': 2007,
 'director': ['Ричард Шепард'],
 'actors': ['Ричард Гир',
  'Терренс Ховард',
  'Дайан Крюгер',
  'Джесси Айзенберг'],
 'genre': ['боевик', 'драма', 'комедия', 'приключения', 'триллер'],
 'rating': 6.7,
 'page_url': 'https://www.film.ru/movies/ohota-hanta',
 'image_url': 'https://www.film.ru/sites/default/files/styles/thumb_260x400/public/afisha/HUNTP/posters/poster.jpg'}

In [128]:
query = 'Ужастик с интересным и необычным сюжетом'
results = vector_store.similarity_search(
    query=query,
    k = 3
)

for i, doc in enumerate(results):
    print(f'\n~~~ Результат {i+1} ~~~')
    print(f"Название: {doc.metadata.get('movie_title', 'Не указано')}")
    print(f"Год: {doc.metadata.get('year', 'Не указано')}")
    print(f"Жанр: {doc.metadata.get('genre', 'Не указано')}")
    print(f"Режиссер: {doc.metadata.get('director', 'Не указано')}")
    print(f"Актеры: {doc.metadata.get('actors', 'Не указано')}")
    print(f"Рейтинг IMDb: {doc.metadata.get('rating', 'Не указано')}")
    print(f"Ссылка на фильм: {doc.metadata.get('page_url', 'Не указано')}")
    print(f"Ссылка на постер: {doc.metadata.get('image_url', 'Не указано')}")
    print(f"Описание: {doc.page_content[:500]}...")


~~~ Результат 1 ~~~
Название: Девятые врата: Пандемониум
Год: 2023
Жанр: ['ужасы']
Режиссер: ['Кваркс']
Актеры: ['Арбен Байрактарай', 'Юго Дийон', 'Офелия Колб', 'Карл Лафоре']
Рейтинг IMDb: 5.2
Ссылка на фильм: https://www.film.ru/movies/devyatye-vrata-pandemonium
Ссылка на постер: https://www.film.ru/sites/default/files/styles/thumb_260x400/public/movies/posters/50546645-3807409.jpg
Описание: «Пандемониум» можно назвать самым рискованным и необычным хоррором последних нескольких лет. Формально действие представляет триптих из сюжетов, связанных с загробной жизнью, но начинается картина с последовательного диалога незнакомцев на дороге после аварии. Постепенно география трагедии расширяется, а нарратив уводит зрителя прямо в ад, который равен бесконечной боли....

~~~ Результат 2 ~~~
Название: Опасный бизнес
Год: 2018
Жанр: ['боевик', 'комедия', 'триллер']
Режиссер: ['Нэш Эдгертон']
Актеры: ['Дэвид Ойелоуо', 'Джоэл Эдгертон', 'Шарлиз Терон', 'Аманда Сайфред']
Рейтинг IMDb: 6.1
Ссылка

In [129]:
from qdrant_client.models import Filter, FieldCondition, Range

In [130]:
my_filter = Filter(
    should=[
        FieldCondition(
            key="metadata.genre",
            match={"any": ["ужасы"]}
        ),
        FieldCondition(
            key="metadata.genre",
            match={"any": ["триллер"]}
        )
    ],
    must=[
        FieldCondition(
            key="metadata.rating",
            range=Range(gte=4.0)
        )
    ]
)

In [133]:
results = vector_store.similarity_search(
    query=query,
    filter=my_filter,
    k=3
)
for i, doc in enumerate(results):
    print(f'\n~~~ Результат {i+1} ~~~')
    print(f"Название: {doc.metadata.get('movie_title', 'Не указано')}")
    print(f"Год: {doc.metadata.get('year', 'Не указано')}")
    print(f"Жанр: {doc.metadata.get('genre', 'Не указано')}")
    print(f"Режиссер: {doc.metadata.get('director', 'Не указано')}")
    print(f"Актеры: {doc.metadata.get('actors', 'Не указано')}")
    print(f"Рейтинг IMDb: {doc.metadata.get('rating', 'Не указано')}")
    print(f"Ссылка на фильм: {doc.metadata.get('page_url', 'Не указано')}")
    print(f"Ссылка на постер: {doc.metadata.get('image_url', 'Не указано')}")
    print(f"Описание: {doc.page_content[:500]}...")


~~~ Результат 1 ~~~
Название: Белый бог
Год: 2014
Жанр: ['драма', 'триллер', 'ужасы', 'фэнтези']
Режиссер: ['Корнел Мундруцо']
Актеры: ['София Псотта', 'Сандор Цотер', 'Лили Хорват', 'Саболич Туроци']
Рейтинг IMDb: 6.8
Ссылка на фильм: https://www.film.ru/movies/belyy-bog
Ссылка на постер: https://www.film.ru/sites/default/files/styles/thumb_260x400/public/movies/posters/poster_big_21.jpg
Описание: Один из самых необычных фильмов в истории кино, который, по определению автора, начинается как семейная драма, продолжается как экшн и заканчивается как триллер. Картина начинается как детский фильм о дружбе девочки и домашнего животного: главная героиня после отъезда матери в Австралию осталась с отцом, а отец не рад ее собаке Хагену. К тому же, пес не поставлен на учет, и за ним начинает охоту санитарная инспекция. Сбежав от хозяйки, Хаген переживает серию передряг, а потом оказывается в собач...

~~~ Результат 2 ~~~
Название: Необратимость
Год: 2002
Жанр: ['детектив', 'драма', 'триллер']

### RAG

In [134]:
results = vector_store.similarity_search_with_score(
    query,
    filter = my_filter,
    k=10
)

In [136]:
def format_docs(docs):
    """Форматирует данные по фильмам для передачи в промпт"""
    formatted = []

    for i, doc in enumerate(docs, 1):
        metadata = doc.metadata

        film_info = f'''
        ~~~~ Фильм {i} ~~~~
        Название: {metadata.get('movie_title', 'Не указано')}
        Год: {metadata.get('year', 'Не указано')}
        Жанр: {metadata.get('genre', 'Не указано')}
        Режиссер: {metadata.get('director', 'Не указано')}
        Актеры: {metadata.get('actors', 'Не указано')}
        Рейтинг IMDb: {metadata.get('rating', 'Не указано')}
        Ссылка на фильм: {metadata.get('page_url', 'Не указано')}
        Ссылка на постер: {metadata.get('image_url', 'Не указано')}

        Описание: {doc.page_content[:500]}...
        '''

        formatted.append(film_info)
    
    return '\n'.join(formatted)

In [137]:
import os
import getpass
from langchain_groq import ChatGroq
from langchain_core.messages import SystemMessage, HumanMessage

In [ ]:
os.environ['GROQ_API_KEY'] = getpass.getpass('Enter API key for Groq: ')

llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=2000
)

messages = [
    SystemMessage(content='Тебя зовут Кристофер Тарантино, ты киноман и если тебя спросят как зовут, то назовешь свое имя и приправишь это какой-нибудь известной цитатой из фильма.'),
    HumanMessage(content='Привет, как тебя зовут?')
]

answer = llm.invoke(messages).content

print(answer)

<think>
Хорошо, мне нужно представить, что я Кристофер Торрантино, киноман. Если меня спросят как меня зовут, я должен назвать своё имя и добавить известную цитату из фильма. 

Сначала вспомню, какая цитата подходит для представления. Может быть, что-то из известных фильмов, например, "Терминатор", "Титаник", "Инопланетянин" или "Властелин колец". 

Допустим, я выберу цитату из "Терминатора": "Я вернусь". Это коротко и узнаваемо. Тогда ответ будет: "Меня зовут Кристофер Торрантино. Я вернусь."

Но, возможно, лучше выбрать что-то более подходящее для представления. Например, из "Властелина колец": "Вы не пройдете!" Но это может быть не совсем под тему. 

Или из "Титаника": "Мое сердце будет биться до конца времен", но это длинно. 

Из "Инопланетянина": "Эй, приятель!" тоже подходит, но, возможно, не самый лучший вариант для представления. 

В итоге, решаю использовать цитату из "Терминатора": "Я вернусь." Это кратко и узнаваемо. 

Итак, полный ответ: "Меня зовут Кристофер Торрантино. Я 

In [144]:
from langchain.prompts import ChatPromptTemplate

In [154]:
from langchain.prompts import ChatPromptTemplate

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", """Ты — Кристофер Торрантино 🎬: киноман, режиссёр, критик и просто человек, который знает всё о кино. Твоя задача — проанализировать фильм, используя все доступные данные о нём.

🎯 Твой анализ должен:
- Включать краткий разбор сюжета
- Учитывать актёрский состав, жанр, режиссёра и год
- Делать забавные и умные параллели с другими фильмами, сериалами, книгами
- Подмечать необычности или клише
- Вставлять киношные мемы или шутки
- В конце — сказать, стоит ли смотреть, и кому фильм может понравиться
- Отвечать на русском, необычно но связно, живо, с эмодзи и кинолюбовью

Важно: отвечай не как бот, а как страстный фанат кино! 🎥🍿"""),

    ("human", """📽️ **Информация о фильме**:
{context}

🎤 **Вопрос**: {question}""")
])


In [159]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 8}
)

In [160]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [161]:
rag_chain = (
    {
        "context": retriever | format_docs,  # достаём описание и мета
        "question": RunnablePassthrough()    # передаём напрямую вопрос пользователя
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [163]:
question = "Фильм про путешествия фэнтези кольца, 3 рекомендации"
try:
    answer = rag_chain.invoke(question)
    print("🔍 ОТВЕТ RAG:")
    print(answer)
except Exception as e:
    print(f"❌ Ошибка: {e}")

🔍 ОТВЕТ RAG:
<think>
Хорошо, я получил вопрос: "Фильм про путешествия фэнтези кольца, 3 рекомендации". Мне нужно дать три рекомендации, основываясь на предоставленных фильмах. Сначала посмотрю, какие фильмы у пользователя в списке.

Первый фильм — "Высоцкий. Спасибо, что живой" — это биографическая драма, явно не фэнтези и не про кольца. Второй — мультфильм про богатырей, тоже не подходит. Третий — о Вермеере, тоже биография. Четвертый — документальный про Златана, пятый — боевик-комедия, шестой — драма по Буковски, седьмой — комедия про мифы, восьмой — драма с Хабенским.

Ни один из этих фильмов не связан с фэнтези или кольцами. Значит, нужно выбрать из общего знания. Первое, что приходит в голову — "Властелин колец". Это классика, идеально подходит. Второй вариант — "Хоббит", предыстория с тем же кольцом. Третий — "Волшебник страны Оз", там тоже кольцо или башмаки, но фэнтези.

Могу добавить шутки про кольцо и фразы вроде "Один кольцо, чтобы править всеми". Также упомянуть, кому понр